In [1]:
# I guessed it would have been easier to upload all the datasets to Google Drive and share it with all the team

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Creating folders for storing temporary jpg files
!mkdir /content/temp
!mkdir /content/temp/test
!mkdir /content/temp/train

In [2]:
%matplotlib inline
!pip install memory_profiler
!pip install path

from memory_profiler import memory_usage
import os
import pandas as pd
from glob import glob
import numpy as np

  Created wheel for memory-profiler: filename=memory_profiler-0.57.0-cp36-none-any.whl size=28992 sha256=fcf129c9b610a7f641c1997a1b31a11d5986685417ab29d483ed6e7ca79bc5a7
  Stored in directory: /root/.cache/pip/wheels/74/20/b5/20964ef97be73d2c3a695c9cad7bccd96d1e3e737a8163861f
Successfully built memory-profiler


In [3]:
# Importing Keras and other pre-processing libraries
from keras import layers
from keras import models
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, LSTM, CuDNNLSTM
from keras import regularizers, optimizers
import keras.backend as K
import librosa
import librosa.display
import pylab
import matplotlib.pyplot as plt
from matplotlib import figure
import gc
from path import Path
from tqdm import tqdm
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12069730566267890265
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13593143633601934987
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 864516450616092029
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15701463552
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14614956311070026955
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


Now it will be time to generate the spectogram for each wav files, we will use the librosa library in order to do so.

In [0]:
def append_ext(fn):
    return fn.rstrip('.wav')+".jpg"

def create_spectrogram(filename,name, folder):
    plt.interactive(False)
    clip, sample_rate = librosa.load(filename, sr=None)
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    S = librosa.feature.melspectrogram(y=clip, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
    filename  = '/content/drive/Shared drives/Deep Learning Gang 👺/images/' + folder + '/' + name + '.jpg'
    plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0)
    plt.close()    
    fig.clf()
    plt.close(fig)
    plt.close('all')
    del filename,name,clip,sample_rate,fig,ax,S

Now we will transform each .wav file from the training set to a corresponding spectogram.


In [0]:
Data_dir=np.array(glob("/content/drive/Shared drives/Deep Learning Gang 👺/wavs_train/*"))

# with tqdm(total=Data_dir.shape[0], position=0, leave=True) as pbar:
#     for file in tqdm(Data_dir, position=0, leave=True):
#         filename,name = file, file.split('/')[-1].split('.')[0]
#         create_spectrogram(filename,name, 'train')
#         pbar.update()

Now we will use ImageDataGenerator, this makes life easier when using a CNN.

In [6]:


traindf=pd.read_csv('/content/drive/Shared drives/Deep Learning Gang 👺/meta/train.csv',dtype=str)
testdf=pd.read_csv('/content/drive/Shared drives/Deep Learning Gang 👺/meta/test.csv',dtype=str)
traindf["fname"]=traindf["fname"].apply(append_ext)
testdf["fname"]=testdf["fname"].apply(append_ext)

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/drive/Shared drives/Deep Learning Gang 👺/images/train/",
    x_col="fname",
    y_col="label",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

valid_generator=datagen.flow_from_dataframe(
    dataframe=traindf,
    directory="/content/drive/Shared drives/Deep Learning Gang 👺/images/train",
    x_col="fname",
    y_col="label",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(64,64))

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 570 invalid image filename(s) in x_col="fname". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 6678 validated image filenames belonging to 41 classes.
Found 2225 validated image filenames belonging to 41 classes.


Now It is time to build our predictive heart, a Convolutional Neural Network.

In [7]:
model = Sequential()

#STACK of Conv Neural Nets
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(41, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

#Fitting keras model, no test gen for now
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=1)

#Evaluate GEnerator
model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 64)       

KeyboardInterrupt: ignored

Now we will test our model.

In [0]:
Test_dir=np.array(glob("/content/drive/Shared drives/Deep Learning Gang 👺/wavs_test/*"))

# for file in tqdm(Test_dir[0:10], total= Test_dir.shape[0]):
#     filename, name = file, file.split('/')[-1].split('.')[0]
#     create_spectrogram(filename, name, 'test')

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory="/content/drive/Shared drives/Deep Learning Gang 👺/images/test/",
    x_col="fname",
    y_col="label",
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=(64,64))

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

test_generator.reset()
loss, acc =  model.evaluate_generator(test_generator, steps=STEP_SIZE_TEST)

print(loss, acc)